<img src="https://ucar-sea.github.io/SEA-ISS-Template/_images/2024_SEA_Logo2.png"
     width="500px"
     alt="NCAR logo"
     style="vertical-align:middle"/>

# Event driven architecture for the IMAP science data center
ISS 2025, presented April 7th

**Authors**:  [Maxine Hartnett](mailto:maxine.hartnett@lasp.colorado.edu) LASP/CU

# Overview
This paper describes some of the design choices and motivations for the infrastructure supporting science data processing and access for the IMAP mission. This infrastructure is built in a cloud-first environment, taking advantage of being developed for the cloud from the beginning. 

![IMAP](images/imap.png)

## Background and Motivation

[IMAP](https://imap.princeton.edu/), or the Interstellar Mapping and Acceleration Probe, is a mission launching in September 2025. It is a deep space heliophysics mission, hosting 10 instruments which will study energetic particles known as Energetic Neutral Atoms (ENAs) and the solar wind, among other things, at L2. 

![IMAP instruments](images/IMAP-instruments.png)

The Science Data Center (SDC), which is run by the Laboratory of Atmospheric and Space Physics (LASP) at CU Boulder, is responsible for the development of low-level processing for all instruments, as well as running all levels of processing, producing valid science files, hosting all files to be accessible by scientists, and sending all relevant data to the heliophysics archive at the Space Physics Data Facility (SPDF), among other things. 

Each instrument has multiple "levels" of data, which indicate data quality and refinement. These range from Level 0, which is raw packets data, all the way through Level 3, which is sutible for scientific research. 

All the processing will occur in the cloud using Amazon web services (AWS), and all code is open sourced and freely available on [Github](https://github.com/IMAP-Science-Operations-Center). 

Since we developed against AWS from the start, we are able to experiment with creating a cloud-first design, rather than having to move existing code or structures into the cloud. There are some significant advantages to developing this way, which will be covered at a high-level within this paper. 

## Cloud advantages and Disadvantages

Usage of cloud based infrastructure, particularly that which is provided by large, for-profit companies such as Amazon, Google, and Microsoft, has been the subject of hot debate in some of the recent scientific software conferences I have attended. Although these tools are well established and widely used, there is still consistent discussion of the pros and cons, particularly for scientific data processing and access. 

Therefore, this will be far from an exhaustive overview of the various pros and cons, or of the many variations between cloud providers. However, neither can I avoid these topics entirely, as it is a relevant question when discussing the choices we made in our infrastructure design, particularly since so many of these design choices would be much more difficult in an on-premise environment. 

Forewarning: the words "processing" and "processes" do a lot of heavy lifting in this section. Here, they are used as a generic name for any code running in service of whatever you're trying to accomplish - so for IMAP, that's things like running our web API, running actual science algorithms, determining when processing should occur, etc. 

When using the cloud for processing, generally, the user is paying for increased convenience. Rather than running servers, staffing people to run those servers, and staffing for security and ongoing maintenance, all those costs are offloaded to Amazon, with a nice cut off the top. In some cases, it isn't worth making that trade, but hiring people to manage all that work is expensive. AWS has massive advantages in nearly every domain because they have so much scale. This scale allows for more efficient use of resources at all levels. 

For example, consider processing on a variable-length input, such as the number of concurrent users of a website. In order to handle the maximum number of users, you need to have 100 servers running your processing code. Otherwise, people get timeouts, and you lose money. However, this load is highly variable. If your website is a shopping website, you're going to have far fewer users in the middle of the night. On the other hand, you may have far more users during heavy shopping periods, such as before Christmas. 

If you needed to run all 100 servers all the time, you would be losing money having to buy upgrades, pay someone to track and fix issues, paying electricity and rent, etc. This is one of the advantages of running in the cloud - you can scale up any time of day, and only pay for what you're using. AWS are the ones running the servers, so you save on all the extra unused processing power you would otherwise need. Even if the servers you are using are a lot more expensive, it starts to make sense quickly. 

What happened to all the unused processing space? AWS sold it to someone else. They don't need to worry (as much) about unused server time, because there is always something else they can schedule in. 

### Scientific Computing in the Cloud

Although scientific processing is generally quite different from running code to process shopping website users (speaking as someone who has done both), within space science we can have periods of high usage and periods where we are sitting idle. IMAP downlinks only occur every three days, so we have bursts of processing, then periods of waiting for the next data downlink. 

In addition, we actually have some advantages when compared to traditional applications. The biggest advantage, in my opinion, is a question of latency. Although scientists get quite cranky if you're too slow to deliver data, it's not really comparable to the sub-second latencies required for handling shopping, online advertising, scheduling, alerting, or some other common use cases. Our latency is measured in days, not milliseconds, and that's a huge advantage in the AWS model because it means we can get the cheaper tiers of nearly everything. AWS is always trying to sell off the extra servers that are not in use at any given time, so they discount any processes which use those resources. 

They also discount smaller processes, which is where scientific computing can be at a disadvantage. The rest of this paper covers this in more detail, so I won't get into it here. 

Finally, the elephant in the room for these discussions - when we talk about space missions, we are necessarily limited by the amount of data we can downlink from the satellite (IMAP is significantly smaller than the entire Earth, and significantly more difficult to connect wires to.) This makes the scale significantly smaller than some other applications which are in the scope of the ISS conference. Over the lifetime of the IMAP mission, we expect a dozen terabytes of data, in total. This generally makes things a lot easier to handle on the infrastructure side of things. 

If you're dealing with a higher volume of data, the decision making for cloud utilization changes. On one hand, having so much data makes running your own servers even more difficult. However, you can start to see some of the advantages of scale at that point, and there are a lot of advantages to minimal movement of that data. While in IMAP we copy files around all over the place, that isn't feasible for some cases. 


# IMAP Design Philosophy

This section will cover some of the tools we use and some of the design philosophy for running repeatable, reliable, maintainable, and fast processing.

## Infrastructure as code

Infrastructure as code is not only a cloud related concept, but it is often found in the cloud space. When running in the cloud, it is all too easy to completely delete all your servers by accident with the press of a button. Ease of bringing up these services can be a double-edged sword - it's also easy to bring them down again, and more than that, pieces of infrastructure tend to prolificate and it's difficult to track where everything ends up. After years of running a system, without some tool in place to ensure it doesn't happen, it can be disastrous trying to determine exactly what servers were running, where, why, and with what configuration. 

This is where infrastructure as code comes in. Every piece of IMAP infrastructure is stored as a piece of code in a repository, and the entire system can be destroyed and recreated within a few hours. This is partly to meet one of our requirements to recover from a total loss within 30 days, but also ensures that the infrastructure side of things is as closely tracked, reviewed, and documented as the rest of the code. The system can get very complex, and having a way to restore previous versions, create test instances in different accounts, and track changes makes running long term a lot simpler. 

Infrastructure as code can be used in on-prem servers too, and I highly recommend it even for simple systems. It's an extra layer of protection, and makes maintenance and migrations so much simpler for everyone. 

It acts like a blueprint for complex infrastructure code, to formally define the structure of the processing code.


<img src="images/blueprint.jpg" alt="House blueprint" width="400"/>


The question of if you should use infrastructure as code is pretty settled (yes, you should!) but the question of tooling remains, as always, up for debate. There are many tools out there for setting up on-prem servers (Ansible, Puppet, and Chef are a few). For setting up cloud infrastructure, the two that we debated were CDK and Terraform. 

Having used both, I can say with confidence that using CDK is a lot easier. It uses an existing language for its base (in our case, we use the Python version) making it powerful but also surprisingly easy to learn. That's not to say it's particularly easy to use, but it does take care of a lot of the difficulties of keeping a remote web of servers, where someone can go in and manually update them, in sync with a mostly static piece of code. The biggest drawback, and the dealbreaker for a lot of people, is that it is only available and only able to set up AWS code. 

Terraform, on the other hand, is part of a class of software known as "cloud agnostic" meaning that, in theory, you could run a piece of terraform code on AWS and on Google's cloud platform or Azure and it would behave the same way. 

In practice, though, making sure your infrastructure as code is cloud agnostic is a lot of work. It also is a lot of work to switch platforms, so it's not something a project should expect to do often. In one of my previous roles, I actually did help migrate a system written in Terraform from AWS to Azure, and we ended up needing to rewrite most of it anyway, because the systems were so different. In my opinion, for most use cases, it's not worth the effort. Most of the work on the IMAP infrastructure has been on general design or on code running inside ephemerial processing containers (called lambda in AWS), which would both transfer to any new provider if we did switch. However, just like the debate on running on the cloud at all, this is heavily debated and highly case-specific. 

## Containerization

We also heavily use containerization using Docker for our services. This allows users to run the same code on individual computers or in the cloud. Similar to the infrastructure-as-code tools, it also explicitly defines requirements for running software to create repeatable and well-defined environments. We use Docker for all our services, which is also well-integrated into AWS. 

## Microservices

What is a lambda, exactly? Lambda is the AWS-specific name for a short running process (maximum of 15 minutes) which can be quickly spun up, ran, and spun down again. These short-running processes form the backbone of a lot of our infrastructure, and they are what is called a _microservice_. 

Microservices became quite popular in the age of cloud computing, partly because of the ease of creating and running these services, and partly because they are quite cheap to run. As small, fast running processes, they are ideal for filling in gaps of real computing in the cloud. This also makes them inexpensive to run for the users.

<img src="images/microservices.png" alt="microservices" width="400"/>

Microservice is a general term for these kinds of smaller processes. They don't all need to be short lived and frequently run, like lambdas. The main key is that each microservice has a limited sphere of responsibility. Just like when building code, there are a lot of advantages to keeping the scope of responsibility small for services. 

Each microservice is a little piece of the factory. Everyone takes something in, builds a very specific piece on, and then hands it forward to the next service. A good rule of thumb for the size of a microservice is that it should be small enough for one developer to rewrite in one day. 

#### Advantages of Microservices:

**Scalability**: With microservices, you are able to scale each piece of the processing separately from every other piece. Within our processing, we start broad when files arrive, to ensure we don't miss any by accident. However, as we go, we decide at various steps not to continue in the pipeline. For every file that arrives, we definitely want to index it in our database. However, we may not want to actually run any processing on that file. Since the indexing and the processing occur at separate places, we can have our lightweight indexer run frequently, but keep the heavier, more expensive servers for processing dormant until they are necessary. 

**Deploy pieces separately**: This is a major advantage during development and troubleshooting. If we update a small piece of the pipeline, we don't need to redeploy the entire thing. Similarly, if one of the steps fails, it's very clear where the bug is inside the code. This helps with maintainability and long-term stability too, since we can be confident the only piece being affected is the one we modified. 

**Separate responsibilities**: Just like how dividing responsibilities clearly within one piece of code using functions and classes results in a piece of code that is clear and easy to understand and reuse, so does dividing the actual services by responsibilities. When each service only has one function, they are easier to string together and far more reusable than one service that does everything in a highly specific way. In the long run, this even could help for reusability across different missions. 

**Running in parallel**: Similar to the scaling question, this helps the overall speed of the system. Everything that can be run in parallel, will be run in parallel. If we have correctly divided pieces into discrete chunks, we should never be running code in series when it could be in parallel. Any task that can run independently in our system, does. Often, in our case, this means each of our ten instruments can run completely separately and in parallel on the same architecture. 

**Easy to replace**: If you need to insert a new step in your processing, or rework something that isn't working, it is easy to swap out pieces of the system as long as you correctly match inputs and outputs. When we determined that the dependency system we had in one of our services was actually far more complicated than what we needed, we pulled it out into a new service. This didn't affect anything except the original service, but now both pieces of code are vastly simplified, and the dependency system is reusable for other tools that need the information. Similarly, we have rewritten whole pieces of code without needing to do extensive refactoring elsewhere. In some ways, it can feel like swapping out pieces of the car while you're driving it, but at least you can do it one steering wheel or pedal at a time, instead of needing to take half the car out!

## Connecting everything together

I have talked a bit about swapping pieces out and matching up APIs, but what does this actually look like? What are we actually passing around? What is used to pass information around? 

This is where queues come in. They act as the wires on the circuit board of our system, passing around packets of data and connecting related services together. They can filter or limit calls to services, sort outputs, and track data as it moves through each service. In our case, we use Amazon Eventbridge, but there are plenty of tools and services to accomplish this, including ones you can run on-prem (Kafka). 

Queues are a great way to solve lots of the problems with moving data around and triggering code to run. However, running them on-prem is expensive and difficult, with a lot of monitoring and tweaking needed to ensure the queue runs smoothly. If you're a small team, or working with small amounts of data, it isn't worth the developer time to get the convenience. However, in AWS, we don't have to worry about those things, as AWS already does them for us! They're cheap to use for the end user, so we can get all the advantages and none of the disadvantages. 

Queues are a key element to ensuring you are never running code and servers when you don't need to, which if you'll remember, is key in cloud development. They are a powerful tool that help wire together anything more complicated than one service directly calling another every time it runs. They can also solve concurrency issues, which is something we did encounter in IMAP - things were a little _too_ parallel, and the same service was getting triggered two times simultaneously. The easy fix was to stick the triggers in a queue, sorted by instrument, so we would only run each process one at a time for each instrument. 


```{figure} images/queues-label.png
---
alt: Pipeline overview
width: 400px
name: pipeline
---
Overview of the IMAP processing pipeline.
```

This is a (simplified) diagram of our processing pipeline which highlights where we use queues (in red.) Queues connect different pieces of architecture together and are a critical part of the overall design. 


## Event Driven Processing

What is actually getting passed around in these queues? What are the real advantages of setting a system up like this? Although, yes, it is nice that things are separate, it probably doesn't seem worth the added complexity and effort. Why not just stick everything in one big server and do it all at once?

For IMAP, a key goal of the overall science is to cross-calibrate different instruments. This means that all the instrument processing is intermingled, with some high levels of one instrument going into the low levels of another instrument. Due to this requirement, we aren't able to process all level 1 data files, then all level 2 data files, then all level 3 data files, but instead need to be more flexible with our processing design. 

```{figure} images/deps.png
---
alt: IMAP dependencies
width: 400px
name: pipeline
---
Overview of the IMAP science file dependencies.
```

Event driven processing can help reduce some of the complexities that are introduced here, especially when we start talking about reprocessing. _Event driven processing_ is a system of design which ties back into the ideas of only running code when you need to. It's all well and good, but how can you decide when you _need_ to run code? What is running through the wires of the queues? What are the microservices passing around? 

It's all events. "Event" is a nice, generic word for a small notification or piece of information which can contain _just_ enough information for the next step in processing. This can be all the information needed, or it could be that the processing then goes to retrieve the other pieces.

In IMAP's case, the very first event is when a file arrives. We get raw binary files directly from the spacecraft, already organized by instrument and day for processing. We call these level 0 files. As soon as a level 0 file arrives, we have a watchdog in AWS which sends an event to our indexer with the filename. The indexer takes this file name, populates our file tracker database, and sends another event to our _batch starter._

The batch starter is the piece of infrastructure which determines if we should run processing. In the case of L0, the main question is, "do we have all the other files necessary for processing?" 

If we do, this is the easy case. Sometimes we only need one file. The batch starter asks our dependency service to determine what files are needed and if we have those files. The dependency checker sends back either a list of files, or a "no, we are missing dependencies for that file." Then, the batch starter assembles all the information needed to run processing (things like, what instrument do we have, what the upstream filenames are, and what date we should generate for) and starts the AWS Batch process which handles all that. 

If we are missing a file, we stop here and wait for the file to arrive. Every arriving file re-triggers this checking process, so as soon as all the files are available we can begin processing. This really reduces the amount of time that we are waiting for all files to arrive, because we can ensure we start processing (in parallel) as soon as every file is available for that given step. 

This ensures that scientific data is available as quickly as possible. In IMAP, we have several instruments that average data across multiple months for their final science. On the other hand, we have instruments that don't need to wait months for the completed data. We can handle both of these cases in the same system in a totally decoupled way, so the data that is quick to process and produce is also quick to be released. 

This event driven system allows us to see immediate results from the overall design of our infrastructure. Our end users are happy to know they are getting data quickly, but we're still able to handle the complex dependency tree of IMAP. 

Although we do see a lot of benefits from this event-driven system, it is not required for a cloud-based system (although it is a popular paradigm). It would be simple to trigger some pieces on a time-based, batch oriented system as well. In fact, we also need to accommodate this case within our system, as we will have larger reprocessing runs that reprocess the entire mission on a fairly regular basis. Just like with the microservices, we are able to mix event-based and time-based processing based on different requirements and use cases. However, event based processing does make it simple to modify systems without worrying about race conditions, and it is a pretty natural fit for the microservices model.


# Disadvantages in this design

## Complexity within infrastructure

Within this design, we have moved a lot of the complexity out from code into infrastructure. Although each piece of the infrastructure is pretty small and isolated, understanding the larger system is very complex. It can also be difficult to track the consequences to changes. Even though we try to limit the number of assumptions in the design stage for each piece of the system, they do by necessity have some assumptions they make, because no one tool has visibility into the entire system. It is simple for a new developer to accidentally violate those assumptions. 

## Limited information is available to each service

Since we are passing around small events, rather than running within some larger system, it is difficult for individual services to access information. We have needed to carefully design shared database tables and other locations for services to retrieve system-wide information from, while also ensuring that there are limited spots where that information is updated, so the information is consistent within a particular run of the whole program. 

## Difficult to get visibility into the entire system

Similarly, it is hard for humans using and working on the SDC to get visibility into the overall health of the system. We have monitoring and tracking set up at various places in the system, and we use these tools to update an expected processing database. However, it's not easy to assemble information from all the different services into one convenient place.


# Advantages of this design

## Decoupling code

Since we are not connecting services directly together, but instead connecting them via queues and other similar structures, we find a huge advantage from decoupling services. In the long term, this makes maintaining, upgrading, and modifying these services much easier. Each service can have its own versions of libraries or tools. It is simple to insert new services in between existing services, or in addition to existing services - if we have a special case for some events, we can filter those events out and send them somewhere else. Alternatively, as long as we match the inputs and outputs, we can insert processing between two steps without modifying either one.

Anyone who has attempted to upgrade the versions of a highly integrated, large, and legacy system can immediately see the advantages of testing, deploying, and upgrading small pieces of a system rather than needing to lift everything at once. Making updates simpler and easier also reduces the need for large jumps in version numbers - it becomes something one person can do in a few days. 

This decoupling helps create a highly flexible and re-workable infrastructure. 

## Scaling

By dividing up processing and running small processes in the cloud, we also scale more effectively within the cloud. One of the major advantages of using the cloud is the ability to scale up as needed. However, scaling up to using larger servers and running longer code is the wrong way to go. Instead, it is preferable to scale horizontally. 

**Vertical scaling**

Vertical scaling is when you commit more resources to process more data. This is the default for most software. If you run code on one file, it takes a certain amount of CPU time and RAM. If you run code on a hundred files, it takes more CPU time and more RAM. Writing parallel code is a perfect example of scaling vertically. Rather than running in series and taking more time, parallelized code runs in parallel and takes more space. On-prem, this is the standard way of running, because you generally want to use all the resources you have available to you. 

**Horizontal scaling**

Horizontal scaling is when you scale by adding more servers for processing. So, instead of having one super strong person to pick up and move your couch, you employ a bunch of children to work together and pick up and move your couch. Code does need to be developed in a specific way to take advantage of horizontal scaling, because you have to scale externally to the code itself. 

Why is horizontal scaling preferable within the cloud? As we previously discussed, cloud providers reward you for using smaller instances. In addition to making them cheaper, these providers can also limit the number of large servers. It is also slower to start up larger servers. Due to all these factors, it is generally preferable to start up a number of small servers rather than one large server. AWS and other providers can also handle horizontal scaling for you, if you are using queues as an input. They will automatically scale up parallel processing until they hit a user-set limit. This can also help avoid some of the manual optimization for server size that is necessary when running in the cloud.

Once everything is set up to scale horizontally, we are only limited by the number of servers we can request, and the cost to run. This allows for significant amounts of scaling in the cloud if needed. 

## End user advantages

Our end users for the IMAP SDC are primarily scientists or instrument engineers. We work closely with them to ensure our system can handle all the processing requirements for all ten instruments. Over the course of development, these requirements have changed extensively, so we have fully tested the flexibility of our design. Fortunately, although we have added many new services, database tables, and queue tooling, the core design has actually remained unchanged from when it was first proposed. This is a testament to the flexibility of the microservices and queuing design. 

In addition to requesting changes within our processing code, we have also been working with our end users to explore data access and visibility into processing. Our main method of data access is a Python wrapper around a REST API (which is described in more detail at another presentation in this conference). During development, scientists are actually able to upload test data into our pipeline without (much) intervention from SDC developers. In this way, we are able to enable scientific development for other users even well before launch. This also allows for a very quick iterative development process on not only the scientific algorithms, but also on the data access methods. This will really enable us to hit the ground running soon after launch with a well-tested and well-understood system for data access. 

The scientists using our systems, and especially those working on instrument science, are very interested in getting data quickly. Since we can scale up as required, and event-driven processing does not delay processing runs unnecessarily, we are able to deliver data quickly to these scientists and make it immediately available in our API. 

Are there any end-user disadvantages? We have definitely encountered some growing pains from this new paradigm. There is less control over when and how processing runs, so having things like release notes or delaying processing because of external information is more difficult. There has also been some difficulty over frequent re-runs of data - since our goal is to have a system which runs without any human input, any time a file is updated, it triggers a cascade of re-processing for all the downstream dependencies. There isn't very much we can do to control this without human input, as it's difficult to determine what has changed and if it's important automatically. However, this does mean we expect to produce more file versions than usual. We will see if that ends up causing further problems with our end users. 


## Onboarding new developers

Using infrastructure as code gives a written, strictly defined record of the required infrastructure for running processing. As long as all this code is well-documented, it is theoretically possible for a new member of the SDC to learn how the entire system works without any help or other information. In practice, it is rather difficult to put together how the entire system works, but thanks to unit tests on the infrastructure, and the ability to roll back any broken changes, it's pretty low-impact for new developers to create and release changes. We can also deploy the entire system in a personal account for more in-depth, manual testing without affecting what is currently running.

Our small, limited responsibility services are also easy to learn and update. It is possible and reasonable to only learn one tiny piece of the infrastructure, which is generally less than a few hundred lines of code. If the changes required are small, and limited to one service, it's very easy to pick up and understand the existing code, even if you don't understand the larger context. 

## Conclusion

In the new paradigm of the cloud, software engineers need to change their thinking to use cloud-based resources to their full capabilities. In the cloud, time is money, which means smaller, quicker running processes are key. To accomplish this, we can use designs like microservices connected via queues and event driven triggering to horizontally scale small, simple processes. These smaller, simpler processes are also easier to maintain, update, and rewrite for long term maintenance. However, we need to carefully manage the information needed at every step in the pipeline, since this divides up the pipeline and no one service has a complete understanding of processing at any given time. 

In order to accomplish this, we use infrastructure as code to codify the complex pipeline in a reproducible and reviewable way. Using queues, we can decouple processing and add filters, parallelization, or new processes between existing services without having to modify the pipeline at large. The queues pass around events, which can then trigger processing as soon as all data is available, as well as limiting the amount of data moving around the entire system. 

All these tools and designs demonstrate the power of running in a cloud-first environment, where we can use all the services provided by AWS to reduce the amount of server maintenance and developer time needed to run our pipeline, while still passing on significant improvements on processing time, scaling and dependency management to our end users.